In [1]:
knowledge_base = {
        "student": [{"concept_id": -2, "type": "academic (primary -> high school)"}],
        "engineer student": [{"concept_id": -1, "type": "academic (college)"}],
        "industrial management engineer": [{"concept_id": 0, "type": "profession"}],
        "industrial management": [{"concept_id": 1, "type": "field"}],
        "software": [{"concept_id": 2, "type": "stuff"}],
        "software engineer": [{"concept_id": 3, "type": "profession"}],
        "software engineer enthusiast": [{"concept_id": 4, "type": "hobby"}],
        "emines": [
            {"concept_id": 5, "type": "institution"},
            {"concept_id": 6, "type": "school"},
        ],
    }


def query_method(s: str):
    resp = knowledge_base.get(s, None)

    if resp is None:
        return []

    return knowledge_base.get(s, None)


In [2]:
text = ("Five years before i was a student now i am an engineer student at EMINES."
        "school of industrial management by the end of this year i will graduate as."
        "An industrial management engineer, I am a software engineer enthusiast nothing to say student and student.")

In [3]:
from skillner.core import Pipeline, Document, Sentence

from skillner.text_loaders import StrTextLoader
from skillner.matchers import SlidingWindowMatcher
from skillner.conflict_resolvers import SpanProcessor

In [4]:
# define extraction pipeline
doc = Document()

extraction_pipeline = Pipeline()

extraction_pipeline.add_node(
    StrTextLoader(text),
    name='loader'
)

extraction_pipeline.add_node(
    SlidingWindowMatcher(
        query_method,
        max_window_size=4,
        pre_filter=lambda word: word.lower()
    ),
    name='matcher'
)

extraction_pipeline.add_node(
    SpanProcessor(
        dict_filters={
            "max_candidate": lambda span: max(span.li_candidates, key=len)
        }
    ),
    name="conflict_resolver"
)

extraction_pipeline.run(doc)

In [5]:
# print matching result as table
for idx, sentence in enumerate(doc, start=1):

    print(f"Sentence {idx}")
    print("-" * len("Sentence 0"))

    print(f"{'position':10} | {'count candidates':20} | {'largest candidate':30}")
    print("-" * (10 + 20 + 30 + 3 * 2))

    for span in sentence.li_spans:
        position = f"({span.start}, {span.stop})"
        count_spans = len(span.li_candidates)

        max_candidate = span.metadata['max_candidate']
        str_max_candidate = " ".join(sentence[max_candidate.window])
        
        print(f"{position:10} | {count_spans:20} | {str_max_candidate:30} ")

    print("=" * (10 + 20 + 30 + 3 * 2))

Sentence 1
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(6, 7)     |                    1 | student                        
(11, 13)   |                    1 | engineer student               
(12, 13)   |                    1 | student                        
(14, 15)   |                    2 | EMINES                         
Sentence 2
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(2, 4)     |                    1 | industrial management          
Sentence 3
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(1, 4)     |                    2 | industrial management engineer 
(7, 10)    |                    3 | software engineer enthusiast   
(13, 14)   |                    1 | student             